In [1]:
import numpy as np

In [2]:
def f(x):
    """the right-hand side of the differential equation"""
    return -6 * np.pi * np.cos(3 * np.pi * x) + 9 * (np.pi**2) * x * np.sin(
        3 * np.pi * x
    )

In [3]:
def u(x):
    """the exact solution of the differential equation"""
    return x * np.sin(3 * np.pi * x)

In [4]:
def gen_val(n, h, a, y):
    """generate the values of the function y at the points x_i"""
    y = np.zeros(n - 1)
    for i in range(n - 1):
        y[i] = -u(a + (i + 1) * h) * (h**2)
    return y

In [5]:
def gen_A(n):
    """generate the matrix A of the linear system of equations Au=p"""
    n = n - 1
    A = np.zeros((n, n))
    for i in range(n):
        A[i, i] = 2
        if i > 0:
            A[i, i - 1] = -1
        if i < n - 1:
            A[i, i + 1] = -1
    return A * ((n + 1) ** 2)


gen_A(8)

array([[128., -64.,   0.,   0.,   0.,   0.,   0.],
       [-64., 128., -64.,   0.,   0.,   0.,   0.],
       [  0., -64., 128., -64.,   0.,   0.,   0.],
       [  0.,   0., -64., 128., -64.,   0.,   0.],
       [  0.,   0.,   0., -64., 128., -64.,   0.],
       [  0.,   0.,   0.,   0., -64., 128., -64.],
       [  0.,   0.,   0.,   0.,   0., -64., 128.]])

In [6]:
def gen_p(n, h, f, a, b):
    """generate the vector p of the linear system of equations Au=p"""
    p = np.zeros((n - 1, 1))
    p[0] = f[0] + a * (n**2)
    for i in range(1, n - 2):
        p[i] = f[i]
    p[n - 2] = f[n - 2] + b * (n**2)
    return p

In [7]:
def gen_matrix(n, h, a, b):
    """
    generate the matrix A and the vector p of the linear system of equations Au=p
    Parameters:
    n: number of intervals
    h: step size
    a: u(0)
    b: u(1)
    """
    A = gen_A(n)
    f_ls = gen_val(n, h, a, f)
    p = gen_p(n, h, f_ls, a, b)
    return A, p

In [8]:
def richardson(A, p, a_k, tol=1e-10, max_iter=100000):
    """
    solve the linear system of equations Au=p using the Richardson iteration method
    Parameters:
    A: matrix A
    p: vector p
    tol: tolerance
    max_iter: maximum number of iterations
    """
    x = np.zeros((A.shape[0], 1))
    for i in range(max_iter):
        r = p - A @ x
        if np.linalg.norm(r) < tol:
            return x, i

        x = x + a_k * r
        if True in np.isnan(x):
            print("x contains nan", i)
            return x, i
    return x, i

In [9]:
def eigenval_k(k, n):
    """
    calculate the spectral radius of the matrix A
    """
    return 4 * n**2 * (np.sin((np.pi * k) / (2 * n))) ** 2

In [10]:
def eigenvals(n):
    """
    calculate the eigenvalues of the matrix A
    """

    return np.array([eigenval_k(k, n) for k in range(1, n)])

In [11]:
a = 0
b = 0
n_ls = np.array([8, 16, 32, 64, 128, 256])

for n in n_ls:
    A, p = gen_matrix(n, np.float32(1 / n), a, b)
    result = richardson(A, p, 0.9 / np.max(eigenvals(n)), tol=1e-10, max_iter=100000)
    print(f"n: {n}, iter: {result[1]}")


n: 8, iter: 122
n: 16, iter: 480
n: 32, iter: 1820
n: 64, iter: 6821
n: 128, iter: 25383
n: 256, iter: 93876
